In [1]:
from wordle_env import WordleEnv, WordleTokens
from stable_baselines3 import PPO
from gymnasium.wrappers import RecordEpisodeStatistics

# calendar: dict[date -> answer]; load from CSV once and reuse
env = RecordEpisodeStatistics(
    WordleEnv(calendar=None, eval_mode=False),  # training
)
model = PPO("MlpPolicy", env, device='cuda', verbose=1, policy_kwargs=dict(
        features_extractor_class=WordleTokens,
        net_arch=[256,256]
    ))
model.learn(total_timesteps=2_500_500)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\darcy\Documents\wordrl\venv\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6        |
|    ep_rew_mean     | 0.216    |
| time/              |          |
|    fps             | 163      |
|    iterations      | 1        |
|    time_elapsed    | 12       |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 5.97       |
|    ep_rew_mean          | 0.0338     |
| time/                   |            |
|    fps                  | 150        |
|    iterations           | 2          |
|    time_elapsed         | 27         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.11115105 |
|    clip_fraction        | 0.522      |
|    clip_range           | 0.2        |
|    entropy_loss         | -4.94      |
|    explained_variance   | 0.0121     |
|    learning_rate        | 0.0003     |
|   

In [10]:
eval_env = WordleEnv(target='actor', eval_mode=True)
obs, _ = eval_env.reset()
done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = eval_env.step(action)

In [11]:
import string
symbols = { -1: "·", 0: "⬜", 1: "🟨", 2: "🟩" }
IDX2LETTER = {i: c for i, c in enumerate(string.ascii_uppercase)}
output = ""
for i in obs:
    word = ""
    tiles = ""
    for j in i:
        if j[1] == -1:
            break
        word += IDX2LETTER[j[0]]
        tiles += symbols[j[1]]
    output += f"{word} {tiles}\n" if tiles != "" else ""
print(output)

ALGAE 🟩⬜⬜⬜⬜
ALOUD 🟩⬜🟨⬜⬜
AROSE 🟩🟨🟨⬜⬜
BATON ⬜🟨🟩🟩⬜
ACTOR 🟩🟩🟩🟩🟩

